In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors
import random


user_profiles = df_interests.groupby("user_id")["interest"].apply(lambda x: "، ".join(x)).reset_index()


guide_tags = df_tags.groupby("guide_id")["tag"].apply(lambda x: "، ".join(x)).reset_index()
guide_profiles = guide_tags.merge(df_guides[["id", "region", "name"]], left_on="guide_id", right_on="id")


vectorizer = TfidfVectorizer()
guide_tfidf = vectorizer.fit_transform(guide_profiles["tag"])


random_index = random.randint(0, len(user_profiles) - 1)
sample_user_id = user_profiles["user_id"].iloc[random_index]
sample_user_interests = user_profiles[user_profiles["user_id"] == sample_user_id]["interest"].values[0]

user_vector = vectorizer.transform([sample_user_interests])
user_info = df_users[df_users["id"] == sample_user_id].iloc[0]


knn = NearestNeighbors(n_neighbors=5, metric="cosine")
knn.fit(guide_tfidf)
distances, indices = knn.kneighbors(user_vector)


recommended_guides = guide_profiles.iloc[indices.flatten()].copy()
recommended_guides["درجة التشابه"] = 1 - distances.flatten()


recommended_guides.insert(0, "المستخدم", user_info["name"])
recommended_guides.insert(1, "اهتماماته", sample_user_interests)
recommended_guides.insert(2, "منطقته", user_info["region"])


recommended_guides = recommended_guides.rename(columns={
    "name": "المرشد المقترح",
    "tag": "اهتمامات المرشد",
    "region": "منطقته (المرشد)"
})


region_boost = 0.05
user_region = user_info["region"]

recommended_guides["الدرجة النهائية"] = recommended_guides.apply(
    lambda row: row["درجة التشابه"] + region_boost if row["منطقته (المرشد)"] == user_region else row["درجة التشابه"],
    axis=1
)


recommended_guides = recommended_guides.sort_values(by="الدرجة النهائية", ascending=False)


display(recommended_guides[[
    "المستخدم", "اهتماماته", "منطقته", "المرشد المقترح", "اهتمامات المرشد", "منطقته (المرشد)", "درجة التشابه", "الدرجة النهائية"
]])
